In [ ]:
%matplotlib notebook
%load_ext rpy2.ipython
from IPython.display import Image
import graphviz
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from os import listdir, makedirs, path, system
import pandas as pd
from pyvis.network import Network
import pydotplus
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from Scripts.read_logs import SolverLogs, get_results
pandas2ri.activate()
readRDS = robjects.r['readRDS']


# Number of nodes
Plot to see what is the upper bound on the number of edges given number of nodes. Useful to see the range of edges for benchmarks.

In [ ]:
def get_num_edges(nodes):
    """Returns number of edges in a complete graph given number of nodes"""
    return nodes * (nodes - 1)/2

def get_num_nodes(edges):
    """Returns number of nodes in a complete graph given number of edges"""
    return 0.5 * (1 + np.sqrt(1 + 8*edges))

num = np.arange(10, 1000, 10)
fig, ax = plt.subplots()
ax.plot(num, get_num_edges(num))
ax.set_title("Complete graph")
ax.set_xlabel("Number of nodes")
ax.set_ylabel("Number of edges")
ax.ticklabel_format(style='sci', axis='y', scilimits=(0,0))

# Visualising networks
There are two ways: with `networkx` module and `pyvis` module.

## networkx
`networkx` gives static plots: 

In [ ]:
fname = "Output/Erdos/E10_N8_I3_M2_S1/carnival_input.dot"

g = nx.drawing.nx_agraph.read_dot(fname)
pos = nx.spring_layout(g, 0.1, iterations=50)

fig, ax = plt.subplots()
nx.draw(g, pos, ax, with_labels=True)

## Pyvis
`pyvis` gives interactive plots:

In [ ]:
g = Network(500, 1000, notebook=True)
g.from_DOT(fname)
g.show(fname.replace(".dot", ".html"))

The method shown below, of going through networkx, allows edges and nodes to be accessible through `g.get_edges()` and `g.get_nodes()` methods

In [ ]:
g = Network(1000, 1000, bgcolor="#414141", notebook=True)
g.from_nx(nx.drawing.nx_agraph.read_dot(fname))
# g.show_buttons()
g.set_options("""
var options = {
  "nodes": {
    "font": {
      "color": "rgba(153,153,153,1)"
    }
  },
  "edges": {
    "color": {
      "inherit": true
    },
    "smooth": false
  },
  "physics": {
    "minVelocity": 0.75
  }
}
""")
g.show(fname.replace(".dot", "2.html"))

## Graphviz

In [ ]:
with open(fname) as fh:
    dot_graph = fh.read()
graphviz.Source(dot_graph)

# Multiple solution gurobi issue

In [ ]:
def get_command(lp_file, **kwargs):
    params = " ".join([f"{x}={y}" for x,y in kwargs.items()])
    return f"gurobi_cl {params} {lp_file}"

dir_name = "Output/Erdos/E1050_N350_I10_M10_S1/"
lp_file = [x for x in listdir(f"{dir_name}/gurobi_N1/") if x.endswith(".lp")][0]
lp_file = f"{dir_name}/gurobi_N1/{lp_file}"
gurobi_params = {
    "MIPGAP": 0.05,
    "PoolGap": 0.0001,
    "SolutionLimit": 500,
    "PoolSolutions": 100,
    "PoolSearchMode": 2
}

for i in range(3):
    sub_dir = f"Output/GurobiMultipleSolutions/NumericFocus{i}"
    if not path.isdir(sub_dir): makedirs(sub_dir)

    command = get_command(lp_file, NumericFocus=i, **gurobi_params,
                          ResultFile=f"{sub_dir}/gurobi_result.sol", 
                          LogFile=f"{sub_dir}/log.txt",
                          SolFiles=f"{sub_dir}/solution")
    system(command)

In [ ]:
sub_dir = "Output/GurobiMultipleSolutions/NumericFocus0/"
df = pd.read_csv(f"{sub_dir}/gurobi_result.sol", sep=" ", skiprows=2, index_col=0, names=["solution"])
df

In [ ]:
def get_obj_value(filename):
    with open(filename) as fh:
        line = fh.readline()
        # skip the first line (Solution for model Obj)
        if "=" not in line: line = fh.readline()
        return float(line.rstrip().split(" = ")[1])

for i in range(3):
    sub_dir = f"Output/GurobiMultipleSolutions/NumericFocus{i}/"
    all_solutions = [f"{sub_dir}/{x}" for x in listdir(sub_dir) if x.endswith(".sol")]
    
    opt_val = get_obj_value(f"{sub_dir}/gurobi_result.sol")
    correct_solutions = [x for x in all_solutions if abs(get_obj_value(x) - opt_val) < 1e-5]
    
    log_num = SolverLogs("gurobi", f"{sub_dir}/log.txt")["solution_count"]
    print(f"Num solutions: {len(correct_solutions)} / {len(all_solutions)}, ", 
          f"Solutions based on log: {log_num}")

In [ ]:
params = {"skiprows": 2, "sep": " ", "index_col": 0}
pd.concat([pd.read_csv(x, **params, names=[f"Solution-{i+1}"]) for i, x in enumerate(correct_solutions)], axis=1)

In [ ]:
%%R

solution <- read.delim("Output/Erdos/E1050_N350_I10_M10_S1/cplex_N1/result_t11_39_53d26_07_2021n85.txt")

solution[, 1] <- as.character(solution[, 1])
idxVarStart <- which(grepl(pattern = "<variables>", x = solution[, 1]))[-1]
idxVarEnd <- which(grepl(pattern = "</variables>", x = solution[, 1]))[-1]

solMatrix <- matrix(data = , nrow = idxVarEnd[1]-idxVarStart[1]-1,
                  ncol = length(idxVarStart))
colnames(solMatrix) <- paste0("Solution-", seq_len(ncol(solMatrix)))
ss1 <- sapply(strsplit(solution[seq(from = idxVarStart[1]+1,
                                  to = idxVarEnd[1]-1, by = 1), 1],
                     split = " "), "[", 5)

rownames(solMatrix) <- sapply((strsplit(ss1, split = "=")), "[", 2)

for(ii in seq_len(ncol(solMatrix))){

    ss1 <-
      sapply(strsplit(solution[seq(from = idxVarStart[ii]+1,
                                   to = idxVarEnd[ii]-1, by = 1), 1],
                      split = " "), "[", 7)
    solMatrix[, ii] <-
      gsub(pattern = "/>", replacement = "",
           x = sapply(strsplit(ss1, split = "="), "[", 2))

}

solMatrix[1:10, ]

In [ ]:
%%R

sol_file <- "Output/GurobiMultipleSolutions/NumericFocus2/gurobi_result.sol"
opt_val <- as.numeric(tail(scan(sol_file, nlines=2, what=character(), quiet=T), 1))
sol_names <- list.files(path=dirname(sol_file), pattern="solution_", recursive=TRUE)

dfs <- read.csv2(sol_file, sep=" ", comment.char="#", header=F, 
                 row.names=1, col.names=c("Names", "Solution-0"))
for (i in 1:length(sol_names)-1) {
    x <- paste0("Output/GurobiMultipleSolutions/NumericFocus2/solution_", i, ".sol")
    obj_val <- as.numeric(tail(scan(x, nlines=2, what=character(), quiet=T), 1))
    if (abs(obj_val - opt_val) < 1e-5) {
        df_temp <- read.csv2(x, sep=" ", comment.char="#", 
                             header=F, row.names=1, col.names=c("Names", paste0("Solution-", i)))
        dfs <- cbind(dfs, df_temp)
    }
}

dfs <- as.matrix(dfs[-1])
dfs[1:10, ]

# Running R code
Running R code in python jupyter notebook

In [ ]:
%%R
print(readRDS("Output/Erdos/E300_N100_I10_M10_S1/cplex_N1/result.Rds")$weightedSIF)